In [ ]:
import requests                # Include HTTP Requests module
from bs4 import BeautifulSoup  # Include BS web scraping module
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey, select
from datetime import date, datetime, timedelta

In [ ]:
def getGamesForDate(games,gameDate):
    df = pd.DataFrame()
    errs = pd.DataFrame()
    n = 0
    for i in games:
        #print(i)
        scores = i.find_all(['span'])
        links = i.find_all(['a'])
        #print(links)
        gameID = ''
        for j in links:
            if j.get_text() == 'BX':
                gameID = j['href'].replace('//','/').split('/')[5].split('?')[0].replace('.xml','')
        try:
            hTm = i.find_all('td')[0].get_text().strip()
        except:
            hTm = ''
        try:
            hSc = int(scores[0].get_text())
        except:
            hSc = 0
        try:
            aTm = i.find_all('td')[2].get_text().strip()
        except:
            aTm = ''
        try:
            aSc = int(scores[1].get_text())
        except:
            aSc = 0
        
        if gameID == '':
            gameID = 'error_' + str(gameDate) + '_' + hTm + '_' + aTm + str(n)
            
        df = pd.concat(
            [df,pd.DataFrame(
                [[gameID,hTm,hSc,aTm,aSc,gameDate,0]]
                , columns = ['ID','home','homeScore','away','awayScore','gameDate','isProcessed'])])
        n += 1
    return df

In [ ]:
dates = []
#https://stackoverflow.com/questions/10688006/generate-a-list-of-datetimes-between-an-interval
def perdelta(start, end, delta):
    curr = start
    while curr < end:
        yield curr
        curr += delta

for result in perdelta(date(2019, 1, 1), date(2020, 1, 1), timedelta(days=1)):
    dates.append(result.strftime("%Y%m%d"))

In [ ]:
gameDate = ''
gameDF = pd.DataFrame()
errorsDF = pd.DataFrame
errors = []

for i in dates:
    gameDate = i
    url = "https://d3hoops.prestosports.com/seasons/men/2019-20/schedule?date=" + gameDate
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")
#soup.prettify()
#soup.find_all('tr', attrs = {'class': ['roster-row1','roster-row0']})
#gameid = url.replace('//','/').split('/')[5].split('?')[0].replace('.xml','')
    games = soup.find_all('tr', attrs = {'class': ['roster-row1','roster-row0']})
    if games:
        gameDF = pd.concat([gameDF,getGamesForDate(games,gameDate)])

In [ ]:
#gameDF.head(10)

engine = create_engine('sqlite:///HoopStat.db')
conn = engine.connect()
metadata = MetaData(bind=None)
games = Table('games', metadata, autoload = True, autoload_with = engine)

for d in dates:
    conn.execute(games.delete(games.c.gameDate == d))

gameDF.to_sql('games', con=engine, if_exists='append', index=False)

#engine.execute("SELECT * FROM games").fetchall()

In [ ]:
conn.close()
#conn.execute(games.delete())
#conn.execute('ALTER TABLE games ADD COLUMN isProcessed bit DEFAULT 0')
#engine.execute("SELECT * FROM games").fetchall()


In [ ]:
gameDF.sort_values(by='ID').head(50)
#gameDF.ID.str.len().max()
#gameDF['isProcessed']